# Zustandsraum für Türme von Hanoi erstellen
- Wir codieren die Beschreibungen durch die Summe der Beschriftungen der Scheiben:
    - Die Scheiben erhalten Zweierpotenzen als Beschriftungen
    - Ein Stab wird als Summe der Beschriftungen der Scheiben codiert
- Wir trennen die Stäbe durch ein Leerzeichen

7 0 0
bedeutet
```
     |        |        |
    |||       |        |
   |||||      |        |
  |||||||     |        |
```

6 0 1
bedeutet
```
     |        |        |
     |        |        |
   |||||      |        |
  |||||||     |       |||
```
4 2 1
bedeutet
```
     |        |        |
     |        |        |
     |        |        |
  |||||||   |||||     |||
```
4 3 0
bedeutet
```
     |        |        |
     |        |        |
     |       |||       |
  |||||||   |||||      |
```

In [13]:
from treelib import Node, Tree
import copy
import queue

STICKS = 3
DISKS = 3

tree = Tree()


# remove top disk from a stick
def remove_top_disk(stick):
    copy.deepcopy(stick)
    if stick == 0:
        return 0, 0

    original_stick = stick

    counter = 0
    while stick & 0b1 != 1:
        counter += 1
        stick = stick >> 1
        if stick & 0b1 == 1:
            break

    counter += 1
    stick = stick >> 1

    stick_without_disk = stick << counter

    return original_stick - stick_without_disk, stick_without_disk


def test_remove_top_disk():
    disk, stick_without_disk = remove_top_disk(7)
    assert 6 == stick_without_disk
    assert 1 == disk

    disk, stick_without_disk = remove_top_disk(6)
    assert 4 == stick_without_disk
    assert 2 == disk

    disk, stick_without_disk = remove_top_disk(5)
    assert 4 == stick_without_disk
    assert 1 == disk

    disk, stick_without_disk = remove_top_disk(4)
    assert 0 == stick_without_disk
    assert 4 == disk

    disk, stick_without_disk = remove_top_disk(3)
    assert 2 == stick_without_disk
    assert 1 == disk

    disk, stick_without_disk = remove_top_disk(2)
    assert 0 == stick_without_disk
    assert 2 == disk

    disk, stick_without_disk = remove_top_disk(1)
    assert 0 == stick_without_disk
    assert 1 == disk

test_remove_top_disk()


def put_disk(stick, disk):
    if disk == 0:
        return False

    if (stick <= disk or remove_top_disk(stick)[0] < disk) and stick != 0:
        return False
    else:
        return True


def test_put_disk():
    assert True == put_disk(0, 1)

    assert True == put_disk(2, 1)

    assert True == put_disk(4, 1)

    assert False == put_disk(1, 1)

    assert False == put_disk(1, 2)

    assert False == put_disk(1, 4)

    assert False == put_disk(1, 6)

test_put_disk()

def contains_disk(stick, disk):
    while stick > 0:
        new_disk, new_stick = remove_top_disk(stick)
        if disk == new_disk:
            return True
        stick = new_stick

    return False

def test_contains_disk():
    assert True == contains_disk(7, 1)
    assert True == contains_disk(5, 1)
    assert False == contains_disk(4, 1)
    assert True == contains_disk(1, 1)

test_contains_disk()

START = "7 0 0"
END = "0 0 7"

q = queue.Queue()
q.put(START)
tree.create_node(START, START)  # Die Ausgangssituation

def expand():
    while not q.empty():
        parent_nid = q.get()

        parent_configuration = [int(x) for x in parent_nid.split(" ")]

        # how can a valid move be determined?
        #  - a larger disk may not lay on top of a smaller one
        #  - a disk below another disk may not be moved
        #  - only one disk may be moved at a time

        # we want to make good moves. Reverting the previous move is nonsensical.

        # generate all possible moves

        # i = source stick
        for i in range(0, STICKS):

            # remove the top disk of the parent's stick
            disk_to_move, new_stick = remove_top_disk(parent_configuration[i])

            # j = target stick
            for j in range(0, STICKS):

                # skip if target and source are equal
                if j == i:
                    continue

                # check if this is a valid move
                if put_disk(parent_configuration[j], disk_to_move):

                    # make a copy
                    child_configuration = copy.deepcopy(parent_configuration)

                    # alter source and destination sticks accordingly
                    child_configuration[i] = new_stick
                    child_configuration[j] += disk_to_move

                    nid = " ".join([str(x) for x in child_configuration])
                    if tree.contains(nid):
                        continue
                    else:
                        if nid == END:
                            print("TARGET REACHED")
                            tree.create_node("## g: " + nid + " ##", nid, parent=parent_nid)
                            return

                        if tree.size() > 1000:
                            print("LIMIT EXCEEDED")
                            return
                        tree.create_node(nid, nid, parent=parent_nid)
                        q.put(nid)


expand()

tree.show()

TARGET REACHED
7 0 0
├── 6 0 1
│   └── 4 2 1
│       ├── 4 3 0
│       │   └── 0 3 4
│       │       ├── 0 2 5
│       │       │   └── 2 0 5
│       │       └── 1 2 4
│       │           └── 1 0 6
│       │               ├── ## g: 0 0 7 ##
│       │               └── 0 1 6
│       └── 5 2 0
└── 6 1 0
    └── 4 1 2
        ├── 4 0 3
        │   └── 0 4 3
        │       ├── 0 5 2
        │       │   └── 2 5 0
        │       │       ├── 2 4 1
        │       │       └── 3 4 0
        │       └── 1 4 2
        │           └── 1 6 0
        │               ├── 0 6 1
        │               └── 0 7 0
        └── 5 0 2

